In [1]:
import os
from datetime import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
import pandas as pd
import seaborn as sns
import talib as ta
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from CNN_Functions_new import *
from DataPrep import *
from CNN_Labels import *
from keras.callbacks import CSVLogger
from decimal import Decimal, getcontext
from sklearn.metrics import confusion_matrix

In [16]:
tickers = ['NVDA','TWTR','MSFT','GOOG','AAPL','TSLA']
#tickers = ['NVDA','TWTR']
dfs = []
testdfs = []
images = []
testimages = []
cnnpred2 = []

for i in range(len(tickers)):
    
    df = pd.DataFrame()
    
    if(tickers[i] == 'AAPL' or tickers[i] == 'BMW'):
        df = pd.read_csv("/Users/rorymurray/Desktop/FYP/Data/"+tickers[i]+".csv", sep =';')
    else:
        df = pd.read_csv("/Users/rorymurray/Desktop/FYP/Data/"+tickers[i]+".csv", sep =',')
        
    df.columns = ['Ticker','Per','Date','Time','Open','High','Low','Close','Vol']
    
    df = prepareData(df)
    trend_labels(df,'Close',window_size = 30,threshold = 5)
    
    split_date = datetime(2020,1,1)
    end_date = datetime(2020,12,31)
    df_test = df.loc[df.index>split_date]
    df_test = df_test.loc[df_test.index<end_date]
    df = df.loc[df.index<split_date]
    
    df,SCdf_test = scale(df,df_test)

    df_x,df_y = create_images(df,15)
    df_testx,df_testy = create_images(SCdf_test,15) 
    
    df.name = tickers[i]
    df_test.name = tickers[i] +'_test'
    
    
    images.append([df_x,df_y])
    testimages.append([df_testx,df_testy])
    dfs.append(df)
    testdfs.append(df_test)
    


Xtrain = np.vstack((images[0][0],images[1][0],images[2][0],images[3][0],images[4][0],images[5][0]))
Ytrain = np.hstack((images[0][1],images[1][1],images[2][1],images[3][1],images[4][1],images[5][1]))
#Xtrain = np.vstack((images[0][0],images[1][0]))
#Ytrain = np.hstack((images[0][1],images[1][1]))

unique, counts = np.unique(Ytrain, return_counts=True)
print('Class inbalance',counts)

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=(3), activation='relu', input_shape=(15,20), padding = 'same'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(filters=64, kernel_size=(3), activation='relu', padding = 'same'))
model.add(MaxPooling1D(pool_size=(2)))


model.add(Flatten())
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation = 'sigmoid'))
    
model.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy'] )
model.summary()

with open('CNNSummary_23.csv','a') as fd:
    model.summary(print_fn=lambda x: fd.write(x + '\n'))
csv_logger = CSVLogger('CNNlog_23.csv', append=True, separator=',')
unique, counts = np.unique(Ytrain, return_counts=True)

weightx = counts[1]/counts[0]
weightx = round(weightx, 2)
getcontext().prec = 2
weighty = float(Decimal(1) - Decimal(weightx))
history = model.fit(Xtrain,Ytrain,epochs=30,verbose=1,class_weight = {0:0.05,1:0.95},callbacks=[csv_logger])



#selldates = [100,50,7]
for i in range(2):
    cnnpred = Test(model,testimages[i][0],testimages[i][1],testdfs[i],window_size=15)
    cnnpred2.append(cnnpred)
    paperTrade(cnnpred,100,10000)

    

trend labels
trend labels
trend labels
trend labels
trend labels
trend labels
Class inbalance [63472  3823]
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 15, 32)            1952      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 7, 32)             0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 7, 64)             6208      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 3, 64)             0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 192)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 965       
____________

In [7]:
BuyHold(testdfs[3])

P&L :  30 %


In [57]:
def CNN(kernal,window,image):
    #tickers = ['NVDA','TWTR','MSFT','GOOG','AAPL','TSLA']
    #tickers = ['NVDA','TWTR','MSFT','GOOG']
    tickers = ['DAL']
    #tickers = ['NVDA','TWTR','JJ','DAL','MSFT','BMW']
    dfs = []
    testdfs = []
    images = []
    testimages = []

    for i in range(len(tickers)):
    
        df = pd.DataFrame()
    
        if(tickers[i] == 'AAPL' or tickers[i] == 'BMW'):
            df = pd.read_csv("/Users/rorymurray/Desktop/FYP/Data/"+tickers[i]+".csv", sep =';')
        else:
            df = pd.read_csv("/Users/rorymurray/Desktop/FYP/Data/"+tickers[i]+".csv", sep =',')
        
        df.columns = ['Ticker','Per','Date','Time','Open','High','Low','Close','Vol']
    
        df = prepareData(df)
        trend_labels(df,'Close',window_size = window,threshold =5)
    
        split_date = datetime(2020,1,1)
        end_date = datetime(2020,12,31)
        df_test = df.loc[df.index>split_date]
        df_test = df_test.loc[df_test.index<end_date]
        df = df.loc[df.index<split_date]
    
        df,SCdf_test = scale(df,df_test)

        df_x,df_y = create_images(df,image)
        df_testx,df_testy = create_images(SCdf_test,image) 
        
    
        df.name = tickers[i]
        df_test.name = tickers[i] +'_test'
    
    
        images.append([df_x,df_y])
        testimages.append([df_testx,df_testy])
        dfs.append(df)
        testdfs.append(df_test)
    


    #Xtrain = np.vstack((images[0][0],images[1][0],images[2][0],images[3][0],images[4][0],images[5][0]))
    #Ytrain = np.hstack((images[0][1],images[1][1],images[2][1],images[3][1],images[4][1],images[5][1]))
    #Xtrain = np.vstack((images[0][0],images[1][0],images[2][0],images[3][0]))
    #Ytrain = np.hstack((images[0][1],images[1][1],images[2][1],images[3][1]))
    Xtrain = images[0][0]
    Ytrain = images[0][1]
    
    #filename = 'HyperParameter.txt'
    #f = open(filename,'a')
    #print('CNN, kernal size :',kernal,' Dense layer :',dense,file=f)
    #print(' ',file=f)
    #f.close()
   
    

    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=(3), activation='relu', input_shape=(image,20), padding = 'same'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Conv1D(filters=64, kernel_size=(3), activation='relu', padding = 'same'))
    model.add(MaxPooling1D(pool_size=(2)))

   
   

   
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy'] )
    model.summary()
    
    tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

    with open('CNNSummaryTrend.csv','a') as fd:
        model.summary(print_fn=lambda x: fd.write(x + '\n'))
    
    
    csv_logger = CSVLogger('CNNlogTrend.csv', append=True, separator=',')
    unique, counts = np.unique(Ytrain, return_counts=True)
    print(counts)

    weightx = counts[1]/counts[0]
    weightx = round(weightx, 2)
    getcontext().prec = 2
    weighty = float(Decimal(1) - Decimal(weightx))
    print(weightx,weighty)
    history = model.fit(Xtrain,Ytrain,epochs=30,verbose=1,class_weight = {0:weightx,1:weighty},callbacks=[csv_logger])

    selldates = [100,50,30]
    #tradeamounts = [3000,6000,10000]
    for i in range(1):
        cnnpred = Test(model,testimages[i][0],testimages[i][1],testdfs[i],window_size=image)
        print('label count',cnnpred['Label'].value_counts())
        for j in range(1):
            paperTrade(cnnpred,selldates[j],10000)
            
    return cnnpred
   

In [58]:
trend = CNN(3,50,25)

trend labels
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_40 (Conv1D)           (None, 25, 32)            1952      
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 12, 32)            0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 12, 64)            6208      
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 6, 64)             0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 384)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 64)                24640     
_________________________________________________________________
dropout_17 (Dropout)         (None, 64) 

In [21]:
window = [30,50,100]
image = [15,25,50]
kernal = [3,6,9]

for j in range(3):
    CNN(3,window[j],image[j])
   

Calculating bullish labels
Calculating bullish labels
Calculating bullish labels
Calculating bullish labels
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_34 (Conv1D)           (None, 15, 32)            1952      
_________________________________________________________________
max_pooling1d_34 (MaxPooling (None, 7, 32)             0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 7, 64)             6208      
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 3, 64)             0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 192)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                1930      
___________

Epoch 1/30
1403/1403 [==============================] - 6s 4ms/step - loss: 0.0271 - accuracy: 0.9886
Epoch 2/30
1403/1403 [==============================] - 6s 4ms/step - loss: 0.0197 - accuracy: 0.9886
Epoch 3/30
1403/1403 [==============================] - 5s 4ms/step - loss: 0.0190 - accuracy: 0.9631
Epoch 4/30
1403/1403 [==============================] - 5s 4ms/step - loss: 0.0166 - accuracy: 0.9013
Epoch 5/30
1403/1403 [==============================] - 6s 4ms/step - loss: 0.0181 - accuracy: 0.8934
Epoch 6/30
1403/1403 [==============================] - 5s 3ms/step - loss: 0.0160 - accuracy: 0.9076
Epoch 7/30
1403/1403 [==============================] - 5s 3ms/step - loss: 0.0169 - accuracy: 0.8907
Epoch 8/30
1403/1403 [==============================] - 5s 4ms/step - loss: 0.0160 - accuracy: 0.9060
Epoch 9/30
1403/1403 [==============================] - 7s 5ms/step - loss: 0.0166 - accuracy: 0.8852
Epoch 10/30
1403/1403 [==============================] - 7s 5ms/step - loss: 0.016

In [5]:
weights = [0.92,0.94,0.96,0.98]
r = 1-weights[0]
r

0.07999999999999996